# Типы данных и их преобразование

Знакомимся с различными типами данных: булевы, числовые, строковые, временные, Nullable, массивы и геокоординаты. Узнаем, как менять тип данных и что такое переполнение типов.

В заданиях используются данные с платформы **Airbnb в Берлине**

Имеются следующие таблицы:

listings – информация о жилье, включая полные описания, характеристики и средние оценки в отзывах; поскольку столбцов очень много, нужные перечислены в текстах самих задач

calendar_summary – информация о доступности и цене того или иного жилья по дням

- listing_id – идентификатор жилья (объявления)
- date – дата
- available – доступность жилья в данный день (t/f)
- price – цена за ночь

reviews – отзывы

- listing_id –  идентификатор жилья  
- id – id отзыва
- date – дата отзыва
- reviewer_id – id ревьюера (автора отзыва)
- reviewer_name – имя автора
- comments – сам отзыв


На Airbnb есть две основные группы пользователей:
- Хозяева (хосты, hosts) – сдают жилье
- Путешественники (travelers) – снимают


1) Начнем с анализа характеристик хозяев в таблице listings! 
Пользователи, сдающие квартиры на Airbnb, зарегистрировались в разное время. Кто-то – очень давно, а кто-то совсем недавно. Давайте проверим, в какой месяц и год зарегистрировалось наибольшее количество новых хостов. В качестве ответа введите дату следующего формата: 2010-12

    SELECT 
        toStartOfMonth(toDateOrNull(host_since)) AS host_since_month,
        uniqExact(host_id) AS count_host_id
    FROM listings  
    GROUP BY  
        host_since_month
    ORDER BY 
        count_host_id DESC 
    LIMIT 1


2) Посмотрим на среднюю частоту ответа среди хозяев (f) и суперхозяев (t).
Значения частоты ответа хранятся как строки и включают значок %, который необходимо заменить на пустоту (''). После этого приведите столбец к нужному типу данных с помощью toInt32OrNull() и посчитайте среднюю частоту отклика в разбивке по тому, является ли хост суперхозяином или нет. В качестве ответа укажите наибольшее среднее.

    SELECT
        AVG(toInt32OrNull(replaceAll(host_response_rate, '%', ''))) AS avg_responce_rate,
        host_is_superhost

    FROM (
        SELECT 
            DISTINCT 
                host_is_superhost,
                host_response_rate,
                host_id

        FROM 
            listings) AS sub 

    GROUP BY  
        host_is_superhost


3) Сгруппируйте данные из listings по хозяевам (host_id) и посчитайте, какую цену за ночь в среднем каждый из них устанавливает (у одного хоста может быть несколько объявлений). Идентификаторы сдаваемого жилья объедините в отдельный массив. Таблицу отсортируйте по убыванию средней цены и убыванию host_id (в таком порядке). В качестве ответа укажите первый массив в результирующей таблице, состоящий более чем из двух id. 

    SELECT 
        host_id,
        AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS avg_price,
        groupArray(id) as array_id
    FROM 
        listings  
    GROUP BY  
        host_id 
    HAVING 
        length(array_id) > 2
    ORDER BY 
        avg_price DESC,
        host_id DESC 
    LIMIT 10


4) Немного усложним предыдущую задачу, и посчитаем разницу между максимальной и минимальной установленной ценой у каждого хозяина. В качестве ответа укажите идентификатор хоста, у которого разница оказалась наибольшей. 

    SELECT 
        host_id,
        MIN(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS min_price,
        MAX(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS max_price,
        (max_price - min_price) AS diff_price
    FROM 
        listings  
    GROUP BY  
        host_id 
    ORDER BY 
        diff_price DESC,
        host_id DESC 
    LIMIT 5


5) Теперь сгруппируйте данные по типу жилья и выведите средние значения цены за ночь, размера депозита и цены уборки. Обратите внимание на тип данных, наличие значка доллара и запятых в больших суммах. Для какого типа жилья среднее значение залога наибольшее?

    SELECT 
        room_type,
        AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS avg_price,
    AVG(toFloat32OrNull(replaceRegexpAll(security_deposit, '[$,]', ''))) AS avg_deposit,
        AVG(toFloat32OrNull(replaceRegexpAll(cleaning_fee, '[$,]', ''))) AS avg_cleaning_fee
    FROM 
        listings  
    GROUP BY  
        room_type 
    ORDER BY 
        avg_deposit DESC


6) В каких частях города средняя стоимость за ночь является наиболее низкой? 
Сгруппируйте данные по neighbourhood_cleansed и посчитайте среднюю цену за ночь в каждом районе. В качестве ответа введите название места, где средняя стоимость за ночь ниже всего.


    SELECT 
        neighbourhood_cleansed,
        AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS avg_price
    FROM 
        listings  
    GROUP BY  
        neighbourhood_cleansed 
    ORDER BY 
        avg_price ASC  


7) В каких районах Берлина средняя площадь жилья, которое сдаётся целиком, является наибольшей? Отсортируйте по среднему и выберите топ-3. 

    SELECT 
        neighbourhood_cleansed,
        AVG(toFloat32OrNull(square_feet)) AS avg_square 
    FROM 
        listings  
    WHERE  
        room_type = 'Entire home/apt'
    GROUP BY  
        neighbourhood_cleansed 
    ORDER BY 
        avg_square DESC   
    LIMIT 3

8) Посмотрим, какая из представленных комнат расположена ближе всего к центру города. В качестве ответа укажите id объявления.

    SELECT 
        id,
        toFloat64OrNull(longitude) as longitude,
        toFloat64OrNull(latitude) as latitude,
        geoDistance(13.4050, 52.5200, longitude, latitude) as distance_id
    FROM 
        listings  
    WHERE  
        room_type = 'Private room'

    ORDER BY 
        distance_id ASC
    LIMIT 1

